# Continuous Control

---




# DDPG Agent 

In [2]:
import numpy as np
import torch
from ENV import environment_loader

In [3]:
env, state_size, action_size, brain_name, num_agents = environment_loader("Reacher_Single/Reacher.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00

In [4]:
def DDPG_eval(agent, env, max_t=100):
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    for i in range(max_t):
        # actions = np.random.randn(num_agents, action_size) # select an action (for each agent); randomly!
        actions = agent.act(states[0]) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [5]:
def DDPG_train(agent, env, max_t=100, num_episodes = 1):
    overall_score = []
    
    for episodes in range(num_episodes):
        env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        
        for i in range(max_t):
            actions = agent.act(states[0]) # select an action (for each agent)
            actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
            env_info = env.step(actions)[brain_name]           # send all actions to the environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent.step(states, actions, rewards, next_states, dones)

            scores += env_info.rewards                         # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        
        returns_in_episode = np.mean(scores)
        overall_score.append(returns_in_episode)
        print('Total score (averaged over agents) this episode: {}'.format(returns_in_episode))
    
    torch.save(new_agent.actor_local.state_dict(), 'checkpoint_actor.pth')
    torch.save(new_agent.critic_local.state_dict(), 'checkpoint_critic.pth')
    return overall_score

In [6]:
%load_ext autoreload
%autoreload 2
from AGENT import DDPG_Agent
import os.path

new_agent = DDPG_Agent(state_size, action_size, 5)

def fileAtLocation(filename, path):
    return os.path.exists(path + filename)

filename="checkpoint_actor.pth"
path = "/"

if fileAtLocation(filename, path): ## NOT WORKING CURRENTLY!
    print("Found previous trained Agent, going to load them!")
    new_agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
    new_agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
overall_score = DDPG_train(new_agent, env, 100, 20)
print(overall_score)

Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.3399999924004078
Total score (averaged over agents) this episode: 0.11999999731779099
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.08999999798834324
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.09999999776482582
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.30999999307096004
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.0
Total score (averaged over agents) this episode: 0.0
Total score (average

In [10]:
new_agent = DDPG_Agent(state_size, action_size, 5)

new_agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
new_agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

DDPG_eval(new_agent, env)

NameError: name 'actor_local' is not defined

In [10]:
env.close()